## Perform XGBoost using 3 fold CV with Optuna(50 trials) and StandardScaler on the Different Linguistic data calculation

In [1]:
!nvidia-smi

Sat Oct  1 23:13:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --quiet optuna

     |████████████████████████████████| 348 kB 4.8 MB/s 
     |████████████████████████████████| 209 kB 70.7 MB/s 
     |████████████████████████████████| 81 kB 11.8 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 147 kB 67.5 MB/s 
     |████████████████████████████████| 112 kB 70.6 MB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 


In [3]:
import numpy as np
import pandas as pd
import optuna as opt
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from pathlib import Path
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
RANDOM_SEED = 2
DATA_DIR = Path("/content/drive/MyDrive/CS760")

In [6]:
Combined = pd.read_parquet(DATA_DIR/"Combined.parquet.snappy")

In [7]:
Combined.shape

(500000, 21)

In [8]:
Combined.columns

Index(['r_stars', 'r_stars_square', 'r_length', 'u_friends_count',
       'u_review_count', 'u_month_age', 'b_stars', 'b_review_count', 'r_sen',
       'r_sub', 'r_rea', 'r_useful', 'r_text', 'processed_txt', 'textblob_sen',
       'vadar_sen', 'textblob_sub', 'textstat_read', 'flesch_read',
       'dale_read', 'cli_read'],
      dtype='object')

### Model 1 : Using textblob sentiment, textblob subjectivity and  flesch kincaid grade as the predictors

In [9]:
x1 = Combined[['textblob_sen','textblob_sub','flesch_read']]
y1 = Combined['r_useful']

X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size=100000, random_state=RANDOM_SEED)


def objective(trial):
  """ Function to tune parameters """
  gc.collect()
  params = {
      "n_estimators":trial.suggest_int('n_estimators', 1, 1001, step=50),
      "max_depth" : trial.suggest_int("max_depth", 2, 20),
      "learning_rate" : trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
      "subsample ": trial.suggest_float("subsample", 0.4, 1, step=0.1),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1, step=0.1),
      "gamma": trial.suggest_float("gamma", 0, 1),
      "min_child_weight": trial.suggest_int("min_child_weight", 1, 31, step=2)
  }

  print("Currently running with:")
  print(params)

  model = xgb.XGBRegressor(objective="reg:squarederror",
                            n_jobs=-1,
                            grow_policy='lossguide',
                            tree_method="gpu_hist",
                            predictor="gpu_predictor",
                            booster='gbtree',
                            sampling_method='gradient_based',
                            eval_metrics=['rmse'],
                            random_state=RANDOM_SEED,
                            enable_categorical=False,
                            **params)
  pipe = make_pipeline(StandardScaler(), model)

  scores = cross_val_score(pipe, X_train, y_train,
    scoring="neg_root_mean_squared_error", cv=KFold(3))

  return -scores.mean()

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
print("Final best parameters:")
study.best_params


[I 2022-10-01 23:14:31,454] A new study created in memory with name: no-name-74e87297-09db-4544-bf6a-19bad1db2174


Currently running with:
{'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample ': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}


[I 2022-10-01 23:14:37,845] Trial 0 finished with value: 4.170444491313877 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}. Best is trial 0 with value: 4.170444491313877.


Currently running with:
{'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample ': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}


[I 2022-10-01 23:14:57,700] Trial 1 finished with value: 4.172208411509144 and parameters: {'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}. Best is trial 0 with value: 4.170444491313877.


...

[I 2022-10-01 23:35:32,692] Trial 48 finished with value: 4.170617953066028 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.08229885714914471, 'subsample': 0.6000000000000001, 'colsample_bytree': 0.4, 'gamma': 0.4616596787607629, 'min_child_weight': 3}. Best is trial 43 with value: 4.17042856733853.


Currently running with:
{'n_estimators': 651, 'max_depth': 3, 'learning_rate': 0.029852887495159474, 'subsample ': 0.8, 'colsample_bytree': 0.4, 'gamma': 0.35613656752290174, 'min_child_weight': 31}


[I 2022-10-01 23:35:40,389] Trial 49 finished with value: 4.170502153821777 and parameters: {'n_estimators': 651, 'max_depth': 3, 'learning_rate': 0.029852887495159474, 'subsample': 0.8, 'colsample_bytree': 0.4, 'gamma': 0.35613656752290174, 'min_child_weight': 31}. Best is trial 43 with value: 4.17042856733853.


Final best parameters:


{'n_estimators': 551,
 'max_depth': 2,
 'learning_rate': 0.027570952029817863,
 'subsample': 0.7000000000000001,
 'colsample_bytree': 0.4,
 'gamma': 0.2748792806697422,
 'min_child_weight': 5}

In [10]:
best_params = study.best_params.copy()

model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          **best_params)
pipe = make_pipeline(StandardScaler(), model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.4, enable_categorical=False,
                              eval_metrics=['rmse'], gamma=0.2748792806697422,
                              grow_policy='lossguide',
                              learning_rate=0.027570952029817863, max_depth=2,
                              min_child_weight=5, n_estimators=551, n_jobs=-1,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=2,
                              sampling_method='gradient_based',
                              subsample=0.7000000000000001,
                              tree_method='gpu_hist'))])

In [11]:
train_pred = pipe.predict(X_train)
test_pred = pipe.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 4.169373151675538, MAE: 1.8406951278670132
test results - RMSE: 4.205674601785969, MAE: 1.844242683005929


### Model 2 : Using vadar sentiment, textblob subjectivity and  dale chall readability as the predictors

In [12]:
x2 = Combined[['vadar_sen','textblob_sub','dale_read']]
y2 = Combined['r_useful']

X_train, X_test, y_train, y_test = train_test_split(x2, y2, test_size=100000, random_state=RANDOM_SEED)

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
print("Final best parameters:")
study.best_params

[I 2022-10-01 23:35:42,738] A new study created in memory with name: no-name-382787ff-ef7c-401d-8a53-744fed63d930


Currently running with:
{'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample ': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}


[I 2022-10-01 23:35:48,530] Trial 0 finished with value: 4.184410267963064 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}. Best is trial 0 with value: 4.184410267963064.


Currently running with:
{'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample ': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}


[I 2022-10-01 23:36:07,238] Trial 1 finished with value: 4.185628690131138 and parameters: {'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}. Best is trial 0 with value: 4.184410267963064.


...

[I 2022-10-01 23:56:22,076] Trial 48 finished with value: 4.184452292928646 and parameters: {'n_estimators': 351, 'max_depth': 2, 'learning_rate': 0.06743163986970109, 'subsample': 0.4, 'colsample_bytree': 0.4, 'gamma': 0.3533514176127199, 'min_child_weight': 25}. Best is trial 41 with value: 4.184294978544556.


Currently running with:
{'n_estimators': 251, 'max_depth': 7, 'learning_rate': 0.026645345469657165, 'subsample ': 0.5, 'colsample_bytree': 0.5, 'gamma': 0.5619614850253246, 'min_child_weight': 29}


[I 2022-10-01 23:56:29,069] Trial 49 finished with value: 4.18502031814434 and parameters: {'n_estimators': 251, 'max_depth': 7, 'learning_rate': 0.026645345469657165, 'subsample': 0.5, 'colsample_bytree': 0.5, 'gamma': 0.5619614850253246, 'min_child_weight': 29}. Best is trial 41 with value: 4.184294978544556.


Final best parameters:


{'n_estimators': 401,
 'max_depth': 3,
 'learning_rate': 0.04264736899360023,
 'subsample': 0.5,
 'colsample_bytree': 0.5,
 'gamma': 0.32771049088968796,
 'min_child_weight': 31}

In [13]:
best_params = study.best_params.copy()

model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          **best_params)
pipe = make_pipeline(StandardScaler(), model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.5, enable_categorical=False,
                              eval_metrics=['rmse'], gamma=0.32771049088968796,
                              grow_policy='lossguide',
                              learning_rate=0.04264736899360023,
                              min_child_weight=31, n_estimators=401, n_jobs=-1,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=2,
                              sampling_method='gradient_based', subsample=0.5,
                              tree_method='gpu_hist'))])

In [14]:
train_pred = pipe.predict(X_train)
test_pred = pipe.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 4.181972227079587, MAE: 1.8525715696616472
test results - RMSE: 4.217486070702622, MAE: 1.8575606261485815


### Model 3 : Using vadar sentiment, textblob subjectivity and coleman liau index as the predictors

In [15]:
x3 = Combined[['vadar_sen','textblob_sub','cli_read']]
y3 = Combined['r_useful']

X_train, X_test, y_train, y_test = train_test_split(x3, y3, test_size=100000, random_state=RANDOM_SEED)

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
print("Final best parameters:")
study.best_params

[I 2022-10-01 23:56:31,179] A new study created in memory with name: no-name-fccc6172-4eab-40f3-a346-fa2e7eaaa9ae


Currently running with:
{'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample ': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}


[I 2022-10-01 23:56:36,995] Trial 0 finished with value: 4.179194592068344 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}. Best is trial 0 with value: 4.179194592068344.


Currently running with:
{'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample ': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}


[I 2022-10-01 23:56:55,944] Trial 1 finished with value: 4.181253151286108 and parameters: {'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}. Best is trial 0 with value: 4.179194592068344.


...

[I 2022-10-02 00:35:26,581] Trial 48 finished with value: 4.190438116724972 and parameters: {'n_estimators': 951, 'max_depth': 7, 'learning_rate': 0.00937934460488247, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.7098362343694936, 'min_child_weight': 7}. Best is trial 23 with value: 4.176112338334519.


Currently running with:
{'n_estimators': 801, 'max_depth': 4, 'learning_rate': 0.00195581865378845, 'subsample ': 1.0, 'colsample_bytree': 0.9, 'gamma': 0.8422767918251292, 'min_child_weight': 19}


[I 2022-10-02 00:35:38,181] Trial 49 finished with value: 4.204909109559069 and parameters: {'n_estimators': 801, 'max_depth': 4, 'learning_rate': 0.00195581865378845, 'subsample': 1.0, 'colsample_bytree': 0.9, 'gamma': 0.8422767918251292, 'min_child_weight': 19}. Best is trial 23 with value: 4.176112338334519.


Final best parameters:


{'n_estimators': 701,
 'max_depth': 3,
 'learning_rate': 0.014298237141647706,
 'subsample': 0.9,
 'colsample_bytree': 0.7000000000000001,
 'gamma': 0.6098120124360216,
 'min_child_weight': 3}

In [16]:
best_params = study.best_params.copy()

model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          **best_params)
pipe = make_pipeline(StandardScaler(), model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.7000000000000001,
                              enable_categorical=False, eval_metrics=['rmse'],
                              gamma=0.6098120124360216, grow_policy='lossguide',
                              learning_rate=0.014298237141647706,
                              min_child_weight=3, n_estimators=701, n_jobs=-1,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=2,
                              sampling_method='gradient_based', subsample=0.9,
                              tree_method='gpu_hist'))])

In [17]:
train_pred = pipe.predict(X_train)
test_pred = pipe.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 4.173407799819726, MAE: 1.8459321267873048
test results - RMSE: 4.210057745049822, MAE: 1.8520084429883956


### Model 4 : Using  vadar sentiment, textblob subjectivity and  flesch reading ease core as the predictors

In [18]:
x4 = Combined[['vadar_sen','textblob_sub','textstat_read']]
y4 = Combined['r_useful']

X_train, X_test, y_train, y_test = train_test_split(x4, y4, test_size=100000, random_state=RANDOM_SEED)

study = opt.create_study(direction='minimize', sampler=TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=50)
print("Final best parameters:")
study.best_params

[I 2022-10-02 00:35:41,662] A new study created in memory with name: no-name-d01497b9-ab77-4ae9-bd33-083ead7d2f2c


Currently running with:
{'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample ': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}


[I 2022-10-02 00:35:47,485] Trial 0 finished with value: 4.177273863292622 and parameters: {'n_estimators': 451, 'max_depth': 2, 'learning_rate': 0.030445460008040798, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.6000000000000001, 'gamma': 0.3303348210038741, 'min_child_weight': 7}. Best is trial 0 with value: 4.177273863292622.


Currently running with:
{'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample ': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}


[I 2022-10-02 00:36:06,420] Trial 1 finished with value: 4.178436439079513 and parameters: {'n_estimators': 651, 'max_depth': 7, 'learning_rate': 0.005249994057689375, 'subsample': 0.8, 'colsample_bytree': 0.7000000000000001, 'gamma': 0.13457994534493356, 'min_child_weight': 17}. Best is trial 0 with value: 4.177273863292622.


...

[I 2022-10-02 00:55:54,300] Trial 48 finished with value: 4.18038993332992 and parameters: {'n_estimators': 551, 'max_depth': 5, 'learning_rate': 0.051119387554078195, 'subsample': 0.5, 'colsample_bytree': 0.8, 'gamma': 0.790762773163753, 'min_child_weight': 25}. Best is trial 42 with value: 4.173262952548937.


Currently running with:
{'n_estimators': 301, 'max_depth': 4, 'learning_rate': 0.1115528917652135, 'subsample ': 0.7000000000000001, 'colsample_bytree': 0.9, 'gamma': 0.5619614850253246, 'min_child_weight': 17}


[I 2022-10-02 00:55:59,005] Trial 49 finished with value: 4.177367826263378 and parameters: {'n_estimators': 301, 'max_depth': 4, 'learning_rate': 0.1115528917652135, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.9, 'gamma': 0.5619614850253246, 'min_child_weight': 17}. Best is trial 42 with value: 4.173262952548937.


Final best parameters:


{'n_estimators': 401,
 'max_depth': 2,
 'learning_rate': 0.1775648688213594,
 'subsample': 0.8,
 'colsample_bytree': 0.7000000000000001,
 'gamma': 0.6349553401244622,
 'min_child_weight': 17}

In [19]:
best_params = study.best_params.copy()

model = xgb.XGBRegressor(objective="reg:squarederror",
                          n_jobs=-1,
                          grow_policy='lossguide',
                          tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          booster='gbtree',
                          sampling_method='gradient_based',
                          eval_metrics=['rmse'],
                          random_state=RANDOM_SEED,
                          enable_categorical=False,
                          **best_params)
pipe = make_pipeline(StandardScaler(), model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbregressor',
                 XGBRegressor(colsample_bytree=0.7000000000000001,
                              enable_categorical=False, eval_metrics=['rmse'],
                              gamma=0.6349553401244622, grow_policy='lossguide',
                              learning_rate=0.1775648688213594, max_depth=2,
                              min_child_weight=17, n_estimators=401, n_jobs=-1,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=2,
                              sampling_method='gradient_based', subsample=0.8,
                              tree_method='gpu_hist'))])

In [20]:
train_pred = pipe.predict(X_train)
test_pred = pipe.predict(X_test)
print(f"train results - RMSE: {mean_squared_error(y_train, train_pred, squared=False)}, MAE: {mean_absolute_error(y_train, train_pred)}")
print(f"test results - RMSE: {mean_squared_error(y_test, test_pred, squared=False)}, MAE: {mean_absolute_error(y_test, test_pred)}")

train results - RMSE: 4.16872315760192, MAE: 1.8424664445900918
test results - RMSE: 4.207492987194956, MAE: 1.848834966430664
